In [2]:
###士兵solders属性attribute    基础属性->单部队战前属性->军团/集结战前属性->军团/集结战时属性
##部队troops   军团corps  阵位battleArray-btay  集结Army
table = {}

# 士兵基础属性basicAttribute   攻击Attack-a，防御Defense-d，连击Combo-c，抵抗Resistance-ac，穿透Penetration-p，抗穿Anti-Penetration-ap，护盾Shield-sd，支援Support-st，军纪MilitaryDiscipline-md

# fmt:off
g_attributes = ['a', 'd',  #攻击Attack-a,  防御Defense-d 
                      'sg', 'sp', 'cr', #攻城Siege, 速度Speed, 负重Carry
                      'c', 'ac',    #连击Combo-c, 抵抗Resistance-ac
                      'p', 'ap',    #穿透Penetration-p, 抗穿Anti-Penetration-ap
                      'sd', 'st', 'md'  #护盾Shield-sd, 支援Support-st, 军纪MilitaryDiscipline-md
                      ]

g_values = [143.0, 92.0,
            0, 40, 13,
            0.15, 0.18, 
            0.11, 0.09, 
            2.45, 0.0, 10.0
        ]

g_bonuses = [
        'generals_fidelity',  #武将忠诚
        'beauty',    #美人
        'generals_interaction',   #武将缘分
        'majesty_talent', #天赋
        'majesty_prestige', #威望
        'Secretgy',   #秘策
        'science',    #科技
        'alliance',   #联盟技能+奇观+帝王赏罚+陈情+城市诏令+官员
        'decrees',    #政令
        'generals_spell', #武将特殊技能
        'defense',    #城池装扮+组合
        'barracks',   #兵营
        'stratagem_att', #武府属性
        'stratagem_strategy', #奇策
        'treasure']   #灵宝

g_city_bonuses = []
# fmt:on


def set_bonus_values(**kwargs):
    bonus = Bonus()
    for key, value in kwargs.items():
        setattr(bonus, key, value)
    bonus.update()
    return bonus._bonus


class City:
    def __init__(self) -> None:
        self.sum = [0.0 for i in range(len(g_attributes))]
        for attr, value in zip(
            g_bonuses,
            [[0 for a in range(len(g_attributes))] for b in range(len(g_bonuses))],
        ):
            # print(attr, value)
            setattr(self, attr, value)

    def set_attr(self, name, values):
        setattr(self, name, values)

    def sum_attributes(self):
        values = [0.0 for i in range(len(g_attributes))]
        for index, a in enumerate(g_attributes):
            # print(index, a)
            for attr in g_bonuses:
                values[index] += getattr(self, attr, 0)[index]
                # print(attr, values)
        return values

    def set_sum(self, values):
        self.sum = values


city = City()


class Bonus:
    def __init__(self):
        self._bonus = []
        for attr, value in zip(g_attributes, [0.0 for i in range(len(g_attributes))]):
            setattr(self, attr, value)
            self._bonus.append(value)
        # print(self._bonus)

    def update(self):
        for index, attr in enumerate(g_attributes):
            self._bonus[index] = getattr(self, attr, 0)
        print("update bonus", self._bonus)


class Solder:  # 士兵
    def __init__(self, values=g_values):
        self.values = values
        for attr, value in zip(g_attributes, values):
            setattr(self, attr, value)


class General:  # 武将
    def __init__(self, L=103, S=103, I=103) -> None:
        self.Leadership = L
        self.Strength = S
        self.Intelligence = I

        self.base_bonus = [0.0 for i in range(len(g_attributes))]
        self.battle_bonus = [0.0 for i in range(len(g_attributes))]


class Flag:
    def __init__(self) -> None:
        self.base_flag = [0.0 for i in range(len(g_attributes))]
        self.battle_flag = [0.0 for i in range(len(g_attributes))]


class Steed:
    def __init__(self, values) -> None:
        self.base_bonus = [values[i] for i in range(len(g_attributes))]
        self.base_spell = [0.0 for i in range(len(g_attributes))]
        self.battle_spell = [0.0 for i in range(len(g_attributes))]


class Troops:  # 部队
    def __init__(self) -> None:
        self.lead_gl = General()
        self.sth_gl = General()
        self.intlg_gl = General()
        self.steed = Steed()
        self.flag = Flag()
        self.solder = Solder()
        self.number = 0

        self.bonus = [0.0 for i in range(len(g_attributes))]
        self.values = [0.0 for i in range(len(g_attributes))]

        for index, a in enumerate(g_attributes):
            self.bonus[index] = (
                self.lead_gl.base_bonus[index]
                + self.sth_gl.base_bonus[index]
                + self.intlg_gl.base_bonus[index]
                + self.steed.base_bonus[index]
                + self.steed.base_spell[index]
                + self.flag.base_flag[index]
                + city.sum[index]
            )
            self.values[index] = self.solder.values[index] * (1 + self.bonus[index])
            print(
                f"{a:<4}, {index:<2}, {self.solder.values[index]:<5}, {self.bonus[index]:<5}, {self.values[index]:<5}"
            )

        for attr, value in zip(g_attributes, self.values):
            setattr(self, attr, value)


class Corps:  # 军团
    def __init__(self) -> None:
        self.tr1 = Troops()
        self.tr2 = Troops()
        self.tr3 = Troops()
        self.tr4 = Troops()
        self.tr5 = Troops()
        self.trSt = Troops()


class Army:
    def __init__(self, *corps) -> None:
        self.cps1 = Corps()
        self.cps2 = Corps()


# 军团/集结战前属性prewarAttribute  护盾=所有部队战前护盾之和，  兵力加权平均。

# 军团/集结战时属性wartimeAttribute  战前属性+战时技能加成

In [ ]:
# g_attributes = ['a', 'd',  #攻击Attack-a,  防御Defense-d 
#                       'sg', 'sp', 'cr', #攻城Siege, 速度Speed, 负重Carry
#                       'c', 'ac',    #连击Combo-c, 抵抗Resistance-ac
#                       'p', 'ap',    #穿透Penetration-p, 抗穿Anti-Penetration-ap
#                       'sd', 'st', 'md'  #护盾Shield-sd, 支援Support-st, 军纪MilitaryDiscipline-md
#                       ]
steed1_Org_69 = Steed([0.4912, 0.7912, 0, 0, 0, 0, 0, 0, 0, 0,0,0]) 
steed2_Org_50 = Steed() 
steed3_Org_50 = Steed() 
steed4_prl_50 = Steed() 
steed5_prl_40 = Steed() 

In [4]:
cityBonus = [4.775, 4.155, 
             0.15, 1.895, 1.342, 
             1.314, 1.385,
             1.558, 0.56,
             1.445, 0, 0]
city.set_sum(cityBonus)
tr = Troops()

a   , 0 , 143.0, 4.775, 825.825
d   , 1 , 92.0 , 4.155, 474.26000000000005
sg  , 2 , 0    , 0.15 , 0.0  
sp  , 3 , 40   , 1.895, 115.8
cr  , 4 , 13   , 1.342, 30.446
c   , 5 , 0.15 , 1.314, 0.3471
ac  , 6 , 0.18 , 1.385, 0.42929999999999996
p   , 7 , 0.11 , 1.558, 0.28137999999999996
ap  , 8 , 0.09 , 0.56 , 0.1404
sd  , 9 , 2.45 , 1.445, 5.990250000000001
st  , 10, 0.0  , 0.0  , 0.0  
md  , 11, 10.0 , 0.0  , 10.0 


In [93]:
city.attributes()

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [94]:
tr = Troops()

a   , 0 , 143.0, 0.0  , 143.0
d   , 1 , 92.0 , 0.0  , 92.0 
sg  , 2 , 0    , 0.0  , 0.0  
sp  , 3 , 40   , 0.0  , 40.0 
cr  , 4 , 13   , 0.0  , 13.0 
c   , 5 , 0.15 , 0.0  , 0.15 
ac  , 6 , 0.18 , 0.0  , 0.18 
p   , 7 , 0.11 , 0.0  , 0.11 
ap  , 8 , 0.09 , 0.0  , 0.09 
sd  , 9 , 2.45 , 0.0  , 2.45 
st  , 10, 0.0  , 0.0  , 0.0  
md  , 11, 10.0 , 0.0  , 10.0 


In [ ]:
print(len(g_attributes))

In [ ]:
import numpy as np

total_solders = 10000
military_discipline = 10

# 军纪    每次交锋损失士兵=最终实际任务*(敌军纪/我军纪)   (min = 1/3, max=1)

In [ ]:
# 伤害damage    原伤害->最终伤害->原实际伤害->最终实际伤害

# 原伤害InitialDamage     攻击、防御、连击、抵抗、克制系数、可参战士兵数
# 最终伤害FinalDamage   原伤害为基础，受相关技能和角阵支援效果影响后得到的伤害。双方技能和角阵支援效果互相抵消后(min=10%)
# 原实际伤害InitialActualDamage     我方穿透及敌方护盾。  =最终伤害*最终穿透 [5,90]，护盾10点抵挡1点最终伤害。
# 最终实际伤害FinalActualDamage      相关技能，互相抵消。[min=0.1]


def InitialDamage():
    intlDmg = 0.0
    return intlDmg


def FinalDamage():
    fnlDmg = 0.0
    return fnlDmg


def InitialActualDamage():
    intlActlDmg = 0.0
    return intlActlDmg


def FinalActualDamage(intlActlDmg):
    finalActualDamage = 0.0
    return fnlActlDmg